# imports

In [17]:
import pandas as pd
import numpy as np

In [4]:
wefarm = pd.read_csv(r"https://producersdirect-backups.s3.eu-west-1.amazonaws.com/athena-results/Unsaved/2025/09/26/b0cd514b-b9cc-4972-a0c2-c91726e6d825.csv")

# peek at the data

In [5]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 50)

wefarm.head(5)

,question_id,question_user_id,question_language,question_content,question_topic,question_sent,response_id,response_user_id,response_language,response_content,response_topic,response_sent,question_user_type,question_user_status,question_user_country_code,question_user_gender,question_user_dob,question_user_created_at,response_user_type,response_user_status,response_user_country_code,response_user_gender,response_user_dob,response_user_created_at
0,3849056,519124,nyn,E ABA WEFARM OFFICES ZABO NIZISHANGWA NKAHI?,NaN,2017-11-22 12:25:03+00,20691011,200868,nyn,E!23 Omubazi Ni Dudu Cipa',NaN,2019-01-24 17:54:06.216221+00,farmer,live,ug,NaN,NaN,2017-11-18 13:09:11+00,farmer,live,ug,NaN,NaN,2017-05-09 09:19:33+00
1,3849061,521327,eng,Q this goes to wefarm. is it possible to get f...,NaN,2017-11-22 12:25:05+00,4334249,526113,eng,Q1 which stage is marleks last vaccinated,NaN,2018-01-04 08:57:28+00,farmer,live,ug,NaN,NaN,2017-11-20 11:55:48+00,farmer,zombie,ug,NaN,NaN,2017-11-22 10:13:03+00
2,3849077,307821,nyn,E ENTE YANJE EZAIRE ENYENA YASHOBERA. \nOBWIRE...,cattle,2017-11-22 12:25:08+00,3849291,296187,nyn,Muhanguzi.Benon kuruga masha isingiro ente yaw...,tomato,2017-11-22 12:35:26+00,farmer,zombie,ug,NaN,NaN,2017-08-22 14:51:07+00,farmer,zombie,ug,NaN,NaN,2017-08-12 09:30:33+00
3,3849077,307821,nyn,E ENTE YANJE EZAIRE ENYENA YASHOBERA. \nOBWIRE...,cattle,2017-11-22 12:25:08+00,3849291,296187,nyn,Muhanguzi.Benon kuruga masha isingiro ente yaw...,cattle,2017-11-22 12:35:26+00,farmer,zombie,ug,NaN,NaN,2017-08-22 14:51:07+00,farmer,zombie,ug,NaN,NaN,2017-08-12 09:30:33+00
4,3849077,307821,nyn,E ENTE YANJE EZAIRE ENYENA YASHOBERA. \nOBWIRE...,cat,2017-11-22 12:25:08+00,3849291,296187,nyn,Muhanguzi.Benon kuruga masha isingiro ente yaw...,tomato,2017-11-22 12:35:26+00,farmer,zombie,ug,NaN,NaN,2017-08-22 14:51:07+00,farmer,zombie,ug,NaN,NaN,2017-08-12 09:30:33+00


In [6]:
wefarm.shape

(20304843, 24)

# install libraries

In [7]:
!pip install gensim pyldavis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 118.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 115.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.6/443.6 kB 44.1 MB/s eta 0:00:00


In [8]:
import nltk
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [9]:
from gensim.parsing.preprocessing import STOPWORDS
from gensim.utils import simple_preprocess
from gensim import models
import gensim

from nltk.stem import WordNetLemmatizer

import pyLDAvis.gensim_models

from tqdm import tqdm

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [10]:
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

# create dataframes by language

In [11]:
wefarm_eng_df = wefarm[wefarm["question_language"] == "eng"][["question_id", "question_content"]].drop_duplicates()

# preprocess text

In [12]:
lem = WordNetLemmatizer()

def preprocess_text(text):
  tokens = simple_preprocess(text, deacc=True)
  return [lem.lemmatize(text, pos='v') for text in tokens if text not in STOPWORDS]

In [13]:
processed_questions_eng_lst = []
for question in tqdm(wefarm_eng_df["question_content"]):
  processed_questions_eng_lst.append(preprocess_text(question))

100%|██████████| 2940542/2940542 [01:12<00:00, 40520.84it/s]


# model

In [14]:
processed_questions_eng_gensim = gensim.corpora.Dictionary(processed_questions_eng_lst)
bow_corpus = []
for doc in tqdm(processed_questions_eng_lst):
  bow_corpus.append(processed_questions_eng_gensim.doc2bow(doc))

tfidf = models.TfidfModel(bow_corpus)
tfidf_corpus = tfidf[bow_corpus]

100%|██████████| 2940542/2940542 [00:10<00:00, 271178.84it/s]


In [20]:
%%time

model_lst = []
for n_t in tqdm(np.linspace(5, 20, 5)):
  model_lst.append(gensim.models.LdaMulticore(tfidf_corpus, num_topics=n_t, id2word=processed_questions_eng_gensim, passes=5))

100%|██████████| 5/5 [31:40<00:00, 380.06s/it]

CPU times: user 29min 31s, sys: 3min 49s, total: 33min 21s
Wall time: 31min 40s


# visualize results

In [21]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(model_lst[0], tfidf_corpus, processed_questions_eng_gensim,  mds="mmds")

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.138763  0.181768       1        1  22.559307
2     -0.005807 -0.233850       2        1  20.899364
1     -0.212793 -0.077966       3        1  19.387379
0     -0.125790  0.216274       4        1  18.649492
3      0.205628 -0.086225       5        1  18.504457, topic_info=        Term          Freq         Total Category  logprob  loglift
219     soil  24005.000000  24005.000000  Default  30.0000  30.0000
25     plant  69154.000000  69154.000000  Default  29.0000  29.0000
52     price  25957.000000  25957.000000  Default  28.0000  28.0000
1160   reply  18121.000000  18121.000000  Default  27.0000  27.0000
1130  follow  17186.000000  17186.000000  Default  26.0000  26.0000
...      ...           ...           ...      ...      ...      ...
82      type   7893.577896  42730.442204   Topic5  -4.9228  -0.0017
21     start   5590.266518  23547.040475   Topic5  -5.2678   0.2492
25     plant   6668.661288  69154.151970   Topic5  -5.0914  -0.6518
36      want   4683.992932  26770.995139   Topic5  -5.4447  -0.0560
149     good   4522.170266  30421.349533   Topic5  -5.4798  -0.2190

[484 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
745       1  0.726198  acre
745       2  0.019008  acre
745       3  0.107554  acre
745       4  0.039787  acre
745       5  0.107554  acre
...     ...       ...   ...
1035      1  0.000441  zero
1035      2  0.001766  zero
1035      3  0.006179  zero
1035      4  0.001324  zero
1035      5  0.990036  zero

[1622 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 3, 2, 1, 4])